<a href="https://colab.research.google.com/github/catasaurus/random-ai-notebooks/blob/main/bart_finetune2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
from transformers import TFAutoModel, BartTokenizer, DefaultDataCollator
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import pandas as pd
import numpy as np
import pyarrow as pa
from tqdm import tqdm
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 19.3 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 34.5 MB/s 
     |████████████████████████████████| 212 kB 77.4 MB/s 
     |████████████████████████████████| 140 kB 68.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.7 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
     |████████████████████████████████| 144 kB 73.8 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |███████████

In [ ]:
model = TFAutoModel.from_pretrained("facebook/bart-base", forced_bos_token_id=0)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
paws = load_dataset('paws', 'labeled_final')
quora = load_dataset('quora')

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartModel.

All the layers of TFBartModel were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartModel for predictions without further training.


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Using custom data configuration default


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

Dataset quora downloaded and prepared to /root/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
quora = pd.DataFrame(quora['train'])
quora['sentence1'] = quora['questions'].apply(lambda x: x['text'][0])
quora['sentence2'] = quora['questions'].apply(lambda x: x['text'][1])
quora = quora.drop('questions', axis=1)
quora = quora.drop('is_duplicate', axis=1)
paws_train = pd.DataFrame(paws['train']).drop('label', axis=1)
dataset = pd.concat([paws_train, quora])
#dataset['sentence1'] = dataset['sentence1'].apply(lambda x: np.array(tokenizer(x)['input_ids']))
#dataset['sentence2'] = dataset['sentence2'].apply(lambda x: np.array(tokenizer(x)['input_ids']))
#dataset_dict = dataset.to_dict(orient='series')
#dataset = {k:dataset_dict[k].tolist() for k in dataset_dict}
del dataset['id']
#dataset = Dataset(pa.Table.from_pydict(dataset))
val_dataset = pd.DataFrame(paws['validation']).drop(['label'], axis=1)
#val_dataset['sentence1'] = val_dataset['sentence1'].apply(lambda x: np.array(tokenizer(x)['input_ids']))
#val_dataset['sentence2'] = val_dataset['sentence2'].apply(lambda x: np.array(tokenizer(x)['input_ids']))

"""
val_dataset_dict = val_dataset.to_dict(orient='series')
val_dataset = {k:val_dataset_dict[k].tolist() for k in val_dataset_dict}
del val_dataset['id']
val_dataset = Dataset(pa.Table.from_pydict(val_dataset))
data_collator = DefaultDataCollator(return_tensors="tf")
val_dataset = val_dataset.to_tf_dataset(columns=["sentence1"],
    label_cols=["sentence2"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
dataset = dataset.to_tf_dataset(columns=["sentence1"],
    label_cols=["sentence2"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)
"""

'\nval_dataset_dict = val_dataset.to_dict(orient=\'series\')\nval_dataset = {k:val_dataset_dict[k].tolist() for k in val_dataset_dict}\ndel val_dataset[\'id\']\nval_dataset = Dataset(pa.Table.from_pydict(val_dataset))\ndata_collator = DefaultDataCollator(return_tensors="tf")\nval_dataset = val_dataset.to_tf_dataset(columns=["sentence1"],\n    label_cols=["sentence2"],\n    shuffle=True,\n    collate_fn=data_collator,\n    batch_size=8,\n)\ndataset = dataset.to_tf_dataset(columns=["sentence1"],\n    label_cols=["sentence2"],\n    shuffle=False,\n    collate_fn=data_collator,\n    batch_size=8,\n)\n'

In [ ]:
#x = np.concatenate((dataset['sentence1'].to_numpy(), dataset['sentence2'].to_numpy()))
#y = np.concatenate((dataset['sentence2'].to_numpy(), dataset['sentence1'].to_numpy()))
#val_x = np.concatenate((dataset['sentence1'].to_numpy(), dataset['sentence2'].to_numpy()))
#val_y = np.concatenate((dataset['sentence2'].to_numpy(), dataset['sentence1'].to_numpy()))
x = dataset['sentence1'].to_numpy()
y = dataset['sentence2'].to_numpy()
val_x = val_dataset['sentence1'].to_numpy()
val_y = val_dataset['sentence2'].to_numpy()
x = tokenizer(list(x), padding=True)['input_ids']
y = tokenizer(list(y), return_tensors='tf', padding=True)['input_ids']
val_x = tokenizer(list(val_x), return_tensors='tf', padding=True)['input_ids']
val_y = tokenizer(list(val_y), return_tensors='tf', padding=True)['input_ids']
padding = [1 for i in range(77)]
for i, j in enumerate(x):
  x[i] = [*list(j), *padding]
x = tf.constant(x)

In [ ]:
model.compile(optimizer='Adam', loss='mse')

In [ ]:
model.fit(x, y, validation_data=(val_x, val_y), epochs=10)
model.save('finetuned_bart.h5')

Epoch 1/10


ValueError: ignored

In [ ]:
y[0]

<tf.Tensor: shape=(275,), dtype=int32, numpy=
array([    0,  1121,   779,   379,  2466,  2156,    37, 16340,  1145,
          19,     5,  2370,  6499,  2156, 13047,  2032,  9144, 22351,
         261,  2156,    11,  2201,  2156,     8,   553,   123,    13,
          10, 12373,     7,   671,     7,  3430,   149,  1156,   479,
           2,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,   